# Бор

Бор — это структура данных для компактного хранения строк.

Он устроен в виде дерева, где на ребрах между вершинами написана символы, а некоторые вершины помечены терминальными. Бор хранит ровно те строки, которые получаются, если выписать подряд все буквы на путях от корня до терминальных вершин. 

![trie](https://koenig-media.raywenderlich.com/uploads/2016/10/SwiftAlgClub_TrieData-trie-1.png)

Бор можно удобно использовать для разных задач:
* Хранение строк — занимает гораздо меньше места, чем массив или сет строк.
* Сортировка строк — по бору можно пройтись dfs-ом и вывести все строки в лексикографическом порядке.

* Сет строк — как мы увидим, в нем легко добавлять и удалять слова, а также проверять, если ли слово в бореи.

## Реализация

Бор состоит из ссылающихся друг на друга вершин. В вершине обычно хранится такая информация:
* терминальная ли вершина,
* ссылки на детей,
* возможно, какая-нибудь дополнительная зависящая от задачи информация о слове, если вершина терминальная. Например, количество таких слов — так можно реализовать мультисет.

In [ ]:
const int k = 26;

struct Vertex {
    Vertex* to[k] = {0};
    bool terminal = 0;
};

Vertex *root = new Vertex();

Чтобы добавить слово в бор, нужно пройти от корня по символам слова. Если перехода по для очередного символа нет — создать его, иначе пройти по уже существующему. Последнюю вершину нужно пометить терминальной.

In [ ]:
void add_string (string &s) {
    v = root;
    for (char c : s) {
        c -= 'a';
        if (!v->to[c]) 
            v->to[c] = new Vertex();
        v = v->to[c];
    }
    v->terminal = true;
}

Чтобы проверить, есть ли слово в боре, нужно пройти от корня по символам слова. Если в конце оказались в терминальной вершине — то есть. Если оказались в нетерминальной или когда-нибудь потребовалось пройтись по несуществущей ссылке — то есть.

Удалить слово можно лениво, просто дойдя до него и убрав флаг терминальности.

### Как хранить ссылки

Хранить ссылки на детей не обязательно в массиве. Возможно, наш алфавит большой — у нас тогда просто не хватит памяти инициализировать столько массивов, большинство из которых будут пустыми.

В этом случае можно придумать какой-нибудь другой способ хранить отображение из символа в ссылку на вершину, например бинарном дереве (`map`) или хэш-таблице (`unordered_map`). Они будут работать дольше (но лишь в константу раз), но зато потребление памяти в них будет линейным. У `map`-а есть ещё одно преимущество, что он хранит ссылки уже отсортированными по символам — так можно отсортировать строки, например.

Учитывайте, что писать бор можно по-разному, особенно когда решаете задачи с жестокими ограничениями.

## Суффиксные ссылки

TODO Когда-нибудь тут появится более подробное описание, а пока почитайте [какую-нибудь другую статью](http://e-maxx.ru/algo/aho_corasick).

Суффиксная ссылка для вершины $v$ — это вершина, в которой оканчивается наидлиннейший собственный суффикс строки, соответствующей вершине $v$ (исключение: для корня бора суффиксная ссылка ведет в себя же).

## Зачем это нужно

> Пусть заданы $n$ *плохих* слов и большой текст $t$. Нужно найти суммарное количество их вхождений в этот текст.

Добавим все плохие слова в бор, и будем считывать строку и с помощью суффиксных ссылок поддерживать самый длинный суффикс текущей строки, который принимает бор.

Тогда, для конкретной позиции, мы можем быстро посчитать, какие плохие слова на нём заканчиваются — ровно те, по которым можно дойти по суффиксным ссылкам. Информацию о количестве таких слов можно посчитать заранее динамикой в графе из суффиксных ссылок.

Помимо суффиксных ссылок, нужно найти ещё *переходы*, чтобы поддерживать самый длинный суффикс.

## Алгоритм Ахо-Корасик*

Заметим, что всего суффиксных ссылок нужно найти $O(n)$, а переходов — $O(nk)$. Суффиксные ссылки и переходы можно быстро найти динамикой.

**Ссылки**. Мы можем сделать так: пройти на символ назад, а оттуда пройти по суффиксной ссылке, и уже оттуда вызвать переход.

**Переходы**. Вот к нашей строке прибавился символ — нужно найти самую длинную строку. Для этого можно откатываться по суффиксным ссылкам, пока не придем в вершину, из которой данный переход есть. Рано или поздно мы либо попадем в такую вершину, либо попадем в корень. Но можно поступить лениво, и просто откатиться на *одну* суффиксную ссылку и взять уже посчитанный переход оттуда.

<img src='https://www.parazitakusok.ru/images/item/5728/item_1109_1510080296_558.jpg' width='250px'>

In [ ]:
const int k = 26;

struct Vertex {
    Vertex *to[k] = {0}, *go[k] = {0};
    Vertex *link = 0, *p;
    int pch;
    Vertex (int _pch, Vertex *_p) { pch = _pch, p = _p; }
};

Vertex *root = new Vertex(-1, 0);

In [ ]:
void add_string (string s) {
    Vertex *v = root;
    for (char _c : s) {
        c -= 'a';
        if (!v->to[c])
            v->to[c] = new Vertex(c, v);
        v = v->to[c];
    }
}

Нам нужно объявить две функции, которые будут ссылаться друг на друга. В интерпретируемых языках (например, в питоне) можно просто объявить две функции, а вот C++ так не умеет — нужно сначала все объявить, а потом ссылаться.

In [ ]:
Vertex* go (Vertex *v, int c);

Vertex* link (Vertex *v) {
    if (!v->link) {
        if (v == root || v->p == root) v->link = root;
        else v->link = go(link(v->p), v->pch);
    }
    return v->link;
}

Vertex* go (Vertex *v, int c) {
    if (!v->go[c]) {
        if (v->to[c]) v->go[c] = v->to[c];
        else if (v == root) v->go[c] = root;
        else v->go[c] = go(link(v), c);
    }
    return v->go[c];
}